In [0]:
spark

Out[3]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.0 
 Master 
 spark://10.139.64.4:7077 
 AppName 
 Databricks Shell

In [0]:
%scala
//mount the blob storage

dbutils.fs.unmount("/mnt/abt")
dbutils.fs.mount(
  source = "wasbs://abtsample@abtsg.blob.core.windows.net/",
  mountPoint = "/mnt/abt",
  extraConfigs = Map("fs.azure.account.key.abtsg.blob.core.windows.net" -> "Y3HcPe9pmuW0Z9x4myTYDNntpkzeGkIrqPfBsGQuoRZRwP2T7VXajQ4FjSsdZoQmCP8av2hMilL5/nQmAN2A0Q=="))

//list files 
dbutils.fs.ls("/mnt/abt")


/mnt/abt has been unmounted.
res12: Seq[com.databricks.backend.daemon.dbutils.FileInfo] = WrappedArray(FileInfo(dbfs:/mnt/abt/Subject_Accelerometer.csv, Subject_Accelerometer.csv, 4893571), FileInfo(dbfs:/mnt/abt/Subject_Program.csv, Subject_Program.csv, 56475))

In [0]:
#read csv data from mount point 
df = spark.read.format('csv').options(header='true').load("/mnt/abt/Subject_Program.csv")


In [0]:
df.head(5)

Out[28]: [Row(SubjectId='719edd16-226c-44a8-9e5f-2be243d5d952', ProgramName='Amplatzer PFO Occluder', Amplitude='3', Frequency='58MHz', Pulse='64'),
 Row(SubjectId='719edd16-226c-44a8-9e5f-2be243d5d952', ProgramName='Quadra Allure MP CRT-P', Amplitude='4', Frequency='44MHz', Pulse='89'),
 Row(SubjectId='719edd16-226c-44a8-9e5f-2be243d5d952', ProgramName='Endurity Pacemaker', Amplitude='7', Frequency='52MHz', Pulse='83'),
 Row(SubjectId='719edd16-226c-44a8-9e5f-2be243d5d952', ProgramName='Engage TR Introducer', Amplitude='5', Frequency='51MHz', Pulse='66'),
 Row(SubjectId='e9001419-63e8-4aab-8fca-99742b998864', ProgramName='Amplatzer PFO Occluder', Amplitude='4', Frequency='46MHz', Pulse='63')]

In [0]:
#If you want to query this data as a table, you can simply register it as a view or a table.
df.createOrReplaceTempView("SubjectProgram")

In [0]:
%sql
select ProgramName , count(*) from SubjectProgram group by ProgramName

ProgramName,count(1)
Engage TR Introducer,200
Endurity Pacemaker,200
Quadra Allure MP CRT-P,200
Amplatzer PFO Occluder,200


In [0]:
%sql
select Pulse , count(*) from SubjectProgram group by Pulse

Pulse,count(1)
69,21
73,14
87,26
64,19
85,22
71,20
98,19
99,23
96,22
100,26


In [0]:
#insert data into sql server 

df.write.format("jdbc").option("url", "jdbc:sqlserver://abtsqlserver1.database.windows.net:1433;databaseName=abtsample").option("dbtable", "dbo.SubjectProgram").option("user", "XXXXXXXX").option("password", "XXXXXXXXX").mode("append").save()